In [1]:
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import torch

2024-02-09 01:10:36.129892: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-09 01:10:36.390417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 01:10:36.390534: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 01:10:36.418387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 01:10:36.497817: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Model Definition

# Modality Specific Module
class ModalitySpecificModule:
    @staticmethod
    def extract_modality_specific_interactions(input_layer):
        bi_gru_output = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128))(input_layer)
        msl_l1 = tf.keras.layers.Dense(128, activation='relu')(bi_gru_output)
        msl_output = tf.keras.layers.Dense(64, activation='relu')(msl_l1)
        return msl_output

    # Input Definition
    text_input = tf.keras.Input(shape=(50, 300))
    audio_input = tf.keras.Input(shape=(50, 5))
    visual_input = tf.keras.Input(shape=(50, 20))

    @staticmethod
    def compute(text_input, audio_input, visual_input):
        # Extract Modality Specific Interactions
        text_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(text_input)
        audio_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(audio_input)
        visual_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(visual_input)
        return text_msm_output, audio_msm_output, visual_msm_output


# Dense Multimodal Fusion Module
class DenseMultimodalFusionModule:
    residual_features = []
    @staticmethod
    def df_block(df_input1, df_input2, df_input3):
        df_output1 = tf.keras.layers.Dense(64, activation='relu')(df_input1)
        df_output2 = tf.keras.layers.Dense(64, activation='relu')(df_input2)
        df_output3 = tf.keras.layers.Dense(64, activation='relu')(df_input3)
        return df_output1, df_output2, df_output3
        
    @staticmethod
    def dense_fusion_layer(f1, f2, f3):
        df_input1 = tf.keras.layers.Concatenate()([f1, f3])
        df_input2 = tf.keras.layers.Concatenate()([f1, f2])
        df_input3 = tf.keras.layers.Concatenate()([f2, f3])
        r = tf.add(tf.add(f1, f2), f3)
        DenseMultimodalFusionModule.residual_features.append(r)
        fusion_output = DenseMultimodalFusionModule.df_block(df_input1, df_input2, df_input3)
        return fusion_output
    
    @staticmethod
    def compute(f1_0, f2_0, f3_0):
        f1_1, f2_1, f3_1 = DenseMultimodalFusionModule.dense_fusion_layer(f1_0, f2_0, f3_0)
        f1_2, f2_2, f3_2 = DenseMultimodalFusionModule.dense_fusion_layer(f1_1, f2_1, f3_1)
        f1_3, f2_3, f3_3 = DenseMultimodalFusionModule.dense_fusion_layer(f1_2, f2_2, f3_2)
        r = tf.add(tf.add(f1_3, f2_3), f3_3)
        DenseMultimodalFusionModule.residual_features.append(r)
        return DenseMultimodalFusionModule.residual_features


# Multimodal Residual Module
class MultimodalResidualModule:
    @staticmethod
    def compute(residual_features):
        final_residual_feature = residual_features[0]
        for r in residual_features[1:]:
            final_residual_feature = tf.add(final_residual_feature, r)
        return final_residual_feature


# Sentiment Classification Module
class SentimentClassificationModule:
    @staticmethod
    def cmumosiRound(element):
        if element < -2:
            result = -3
        elif -2 <= element and element < -1:
            result = -2
        elif -1 <= element and element < 0:
            result = -1
        elif 0 <= element and element <= 0:
                result = 0
        elif 0 < element and element <= 1:
                result = 1
        elif 1 < element and element <= 2:
                result = 2
        elif element > 2:
                result = 3
        return result
    
    @staticmethod
    def convertLabels(label):
        if label < -2:
            result = -3
        elif -2 <= label and label < -1:
            result = -2
        elif -1 <= label and label < 0:
            result = -1
        elif 0 <= label and label <= 0:
                result = 0
        elif 0 < label and label <= 1:
                result = 1
        elif 1 < label and label <= 2:
                result = 2
        elif label > 2:
                result = 3

        rep = {
            -3 : [1., 0., 0., 0., 0., 0., 0.],
            -2 : [0., 1., 0., 0., 0., 0., 0.],
            -1 : [0., 0., 1., 0., 0., 0., 0.],
            0 : [0., 0., 0., 1., 0., 0., 0.],
            1 : [0., 0., 0., 0., 1., 0., 0.],
            2 : [0., 0., 0., 0., 0., 1., 0.],
            3 : [0., 0., 0., 0., 0., 0., 1.]
        }
        
        return torch.tensor(rep[result])

    @staticmethod
    def compute(residual_feature):
        output_l1 = tf.keras.layers.Dense(64, activation='relu')(residual_feature)
        output_l2 = tf.keras.layers.Dense(32, activation='relu')(output_l1)
        output_l3 = tf.keras.layers.Dense(16, activation='relu')(output_l2)
        output_l4 = tf.keras.layers.Dense(8, activation='relu')(output_l3)

        num_classes = 7
        sentiment = tf.keras.layers.Dense(num_classes, activation='softmax')(output_l4)
        return sentiment


text_input, audio_input, visual_input = ModalitySpecificModule.text_input, ModalitySpecificModule.audio_input, ModalitySpecificModule.visual_input
text_msm_output, audio_msm_output, visual_msm_output = ModalitySpecificModule.compute(text_input, audio_input, visual_input)
residual_features = DenseMultimodalFusionModule.compute(text_msm_output, audio_msm_output, visual_msm_output)
final_residual_feature = MultimodalResidualModule.compute(residual_features)
output = SentimentClassificationModule.compute(final_residual_feature)

model = tf.keras.Model(inputs=[text_input, audio_input, visual_input], outputs=output)

model.summary()

2024-02-09 01:10:43.097990: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-09 01:10:43.102472: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-09 01:10:43.102783: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 300)]            0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 50, 5)]              0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 50, 20)]             0         []                            
                                                                                                  
 bidirectional (Bidirection  (None, 256)                  330240    ['input_1[0][0]']             
 al)                                                                                          

In [3]:
model.compile(
  optimizer='adam', 
  loss=tf.keras.losses.MeanAbsoluteError(), 
  metrics=[
    tf.keras.metrics.Accuracy(),
    tf.keras.metrics.F1Score()
  ]
)

In [4]:
with open('mosi_data.pkl', 'rb') as fp:
    data = pickle.load(fp)

In [5]:
print(data.keys())
print(data['train'].keys())
print(data['train']['vision'].shape)
print(data['train']['audio'].shape)
print(data['train']['text'].shape)
print(data['train']['labels'].shape)
print(data['train']['id'][0].shape)

dict_keys(['valid', 'test', 'train'])
dict_keys(['vision', 'labels', 'text', 'audio', 'id'])
(1284, 50, 20)
(1284, 50, 5)
(1284, 50, 300)
(1284, 1, 1)
(3,)


In [7]:
def prepare_data(type):
    text_input = data[type]['text']
    audio_input = data[type]['audio']
    visual_input = data[type]['vision']
    labels = [SentimentClassificationModule.convertLabels(element) for element in data[type]['labels'][:, 0][:, 0]]
    labels = np.array(labels)
    return text_input, audio_input, visual_input, labels

train_text_input, train_audio_input, train_visual_input, train_labels = prepare_data('train')
valid_text_input, valid_audio_input, valid_visual_input, valid_labels = prepare_data('valid')

In [8]:
training_dataset = [train_text_input, train_audio_input, train_visual_input]
validation_data = ([valid_text_input, valid_audio_input, valid_visual_input], valid_labels)

In [10]:
model.fit(training_dataset, train_labels, validation_data=validation_data, epochs=50)

Epoch 1/50


2024-02-09 01:12:16.563614: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:542] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2024-02-09 01:12:16.566532: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:574 : INTERNAL: libdevice not found at ./libdevice.10.bc
2024-02-09 01:12:16.613826: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:542] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2024-02-09 01:12:16.615930: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:574 : INTERNAL: libdevice not found at ./libdevice.10.bc
2024-02-09 01:12:16.662627: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:542] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2024-02-09 01:12:16.664643: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:574 

InternalError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_57 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/christos/Projects/MSA/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/christos/Projects/MSA/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/christos/Projects/MSA/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/christos/Projects/MSA/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/home/christos/Projects/MSA/.venv/lib/python3.11